# 中文分词简介

## 1、HMM 分词  

HMM 分词属于由字构词的分词方法，将分词问题转化为序列标注问题。该方法不依赖先验的分词词库，但需要分好词的语料进行训练。

每个字有4个可能的状态：词首 B，词中 M，词尾 E，单字成词 S  


### 训练过程

训练时需要统计的信息：状态初始概率 $Pi$、转移矩阵 $H$ 和发射矩阵 $E$  

在 HMM 中每个参数经统计语料得到的：  

- 状态初始概率 $\pi_i$ 就是语料中每一行文本第一个字属于 B、M、E、S 的概率  

```
{
	'B': 0.5820,
	'S': 0.4180,
	'E': 0.0,
	'M': 0.0
}
```

- 转移矩阵 $H$ 是语料文本中上一个状态转移到下一个状态的概率  

```
{
	'B': {
		'B': 0.0,
		'S': 0.0,
		'E': 0.8833,
		'M': 0.1167
	},
	'S': {
		'B': 0.3608,
		'S': 0.4710,
		'E': 0.0,
		'M': 0.0
	},
	'E': {
		'B': 0.4695,
		'S': 0.5305,
		'E': 0.0,
		'M': 0.0
	},
	'M': {
		'B': 0.0,
		'S': 0.7222,
		'E': 0.0,
		'M': 0.2778
	}
}
```

- 发射矩阵 $E$ 是语料文本中每个字属于某个状态的概率  

```
{
	'B': {
		'竖': 3.6e-06,
		'岛': 2.95e-05,
		'陷': 7.27e-05,
		'洪': 4.46e-05,
		'鸦': 5.76e-06,
        ...
	}),
	'S': {
		'嫂': 7.90e-06,
		'岛': 0.00,
		'陷': 1.69e-05,
		'龚': 1.91e-05,
		'鸦': 2.82e-06,
        ...
	}),
	'E': {
		'岛': 0.00,
		'陷': 3.91e-05,
		'洪': 1.05e-05,
		'单': 0.00,
		'闻': 0.00,
        ...
	}),
	'M': {
		'岛': 5.79e-05,
		'陷': 5.79e-05,
		'洪': 9.35e-05,
		'单': 0.008,
		'闻': 0.00,
        ...
	})
}
```

### 分词过程  

输入为待分词的文本 $\displaystyle y_{1},\dots ,y_{T}$。 分词的过程就是通过训练的带的参数，求出输入对应的最有可能的状态序列的过程。  

**eg: 待分词文本是 "新华网驻东京记者报道"**

1、先求第一个字对应状态的概率  

$V_{1,k} = P(y_1|k) * \pi_k, k \in \{B,M,E,S\}$ 其中 $y_1$ 就是待分词序列的第一个字，$P(y_1|k)$ 可通过发射概率矩阵 $E$ 得到，记录每个位置属于某个状态的概率和到达当前状态的路径  

```
V: [{
	'B': 0.0036,  # 当前位置状态为 'B'的概率
	'M': 0.0,
	'S': 0.0011,
	'E': 0.0
}]

path: {
	'B': ['B'],  # 到达当前位置状态 'B'对应的路径，因为这个是第一个字，所以路径中暂时一个状态，就是本身
	'M': ['M'],
	'S': ['S'],
	'E': ['E']
}

通过概率 max(V) 就可以从 path 中得到状态路径

```

2、求从第二个字到最后一个字对应的状态概率  

$V_{t,k} = {max}_{x \in \{B,M,E,S\}}(P(y_t|k) * H_{x,k} * V_{t-1,x}), k \in \{B,M,E,S\}$, 其中 $V_{t,k}$ 是前 $t$个最终状态为 $k$ 的观测结果最有可能对应的状态序列的概率, $x$ 是前一个字对应的状态， $k$ 是当前字可能的状态，通过不断递归可求解出整句话对应的状态序列路径。

```
V: [{
	'B': 0.00368,
	'M': 0.0,
	'S': 0.001,
	'E': 0.0
}, {
	'B': 1.970e-07,
	'M': 1.309e-05,
	'S': 3.924e-07,
	'E': 2.077e-06
}, {
	'B': 1.222e-10,
	'M': 0.0,
	'S': 1.382e-09,
	'E': 8.230e-12
}, {
	'B': 1.224e-13,
	'M': 0.0,
	'S': 1.866e-13,
	'E': 5.634e-15
}, {
	'B': 6.270e-17,
	'M': 3.038e-17,
	'S': 3.870e-17,
	'E': 8.71e-17
}, {
	'B': 2.799e-21,
	'M': 2.549e-20,
	'S': 9.729e-21,
	'E': 1.511e-19
}, {
	'B': 2.159e-22,
	'M': 3.030e-24,
	'S': 2.094e-23,
	'E': 1.955e-24
}, {
	'B': 0.0,
	'M': 4.941e-27,
	'S': 1.581e-26,
	'E': 9.9775e-25
}, {
	'B': 1.270e-27,
	'M': 3.181e-31,
	'S': 2.795e-28,
	'E': 0.0
}, {
	'B': 7.845e-32,
	'M': 2.095e-31,
	'S': 5.775e-32,
	'E': 3.048e-30
}]


path: {
	'B': ['B', 'M', 'S', 'S', 'B', 'E', 'B', 'E', 'S', 'B'],
	'M': ['B', 'M', 'S', 'S', 'B', 'E', 'B', 'E', 'B', 'M'],
	'S': ['B', 'M', 'S', 'S', 'B', 'E', 'B', 'E', 'S', 'S'],
	'E': ['B', 'M', 'S', 'S', 'B', 'E', 'B', 'E', 'B', 'E']
}

```

在每个位置 $t$ 处，都计算出了该位置每个可能状态的概率，并且也对应计算出了到达该位置对应状态的最大概率路径  

3、求出最终分词状态路径  

${state}_t = arg max(V[t][k]), k \in \{B,M,E,S\}, {path}_{end} = path[{state}_t]$, 其中$t$ 是待分词序列最后一个字的位置，即计算出该位置最可能的状态 ${state}_t$, ${path}_{end}$ 就是到达状态 ${state}_t$对应的路径。

4、输出分词结果  

```
新华网驻东京记者报道  

['B', 'M', 'E', 'S', 'B', 'E', 'B', 'E', 'B', 'E']  

新华网/驻/东京/记者/报道
```

## 2、结巴分词  

#### 1、粗分句  

使用正则表达式对输入的文本进行分句，以便可以输入很长的文本  

```
re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%]+)", re.U)
re_han_cut_all = re.compile("([\u4E00-\u9FD5]+)", re.U)

blocks = re_han.split(sentence)

```

#### 2、细分句  

对粗分产生的 blocks 中不能被 re.han 匹配的会直接作为结果返回，能匹配的进入下个阶段，进行进一步的分词。  

```
for blk in blocks:
    if not blk:
        continue
    if re_han.match(blk):
        for word in cut_block(blk):
            yield word
    else:
        tmp = re_skip.split(blk)
        for x in tmp:
            if re_skip.match(x):
                yield x
            elif not cut_all:
                for xx in x:
                    yield xx
            else:
                yield x
```

#### 3、构建 DAG  

```
def get_DAG(self, sentence):
    self.check_initialized()
    DAG = {}
    N = len(sentence)
    for k in xrange(N):
        tmplist = []
        i = k
        frag = sentence[k]
        while i < N and frag in self.FREQ:
            if self.FREQ[frag]:
                tmplist.append(i)
            i += 1
            frag = sentence[k:i + 1]
        if not tmplist:
            tmplist.append(k)
        DAG[k] = tmplist
    return DAG
```

**eg. 新华网驻东京记者报道**  

```
DAG:{
	0: [0, 1, 2],
	1: [1],
	2: [2],
	3: [3],
	4: [4, 5],
	5: [5],
	6: [6, 7],
	7: [7],
	8: [8, 9],
	9: [9]
}
```

基于用户自定义词库构建前缀词典，对输入文本进行切分，使用 dict 存储切分得到的图数据结构，字典中的 key 是句子中每个字的 index, value 是一个 list，是该字可达的分词路径。比如{0：[0，1,2]}意思就是“新”，“新华”和“新华网”这三个词在词典中都存在。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/seg/jieba.jpg?raw=true)
<center> **结巴分词 DAG 图** </center >

#### 4、求解最大概率路径  

从 DAG 图可以看到，有多种不同的路径可以进行分词，所以需要修改最大概率路径进行最终的分词，jieba 分词采用反向计算最大概率路径的方式进行求解，从后往前计算出每个节点到其它节点的最大概率值及路径。  

```
def calc(self, sentence, DAG, route):
    N = len(sentence)
    route[N] = (0, 0)
    logtotal = log(self.total)    # 词库中所有词的词频数总和，取log 可以避免数值下溢
    for idx in xrange(N - 1, -1, -1):
        route[idx] = max((log(self.FREQ.get(sentence[idx:x + 1]) or 1) -
                          logtotal + route[x + 1][0], x) for x in DAG[idx])
```

```
{
	0: (-46.58362894518131, 2),  # 节点 0 到 2 的概率最大
	1: (-54.21474216683713, 1),  # 节点 1 到 1 的概率最大
	2: (-44.684333386034865, 2), # 节点 2 到 2 的概率最大
	3: (-35.655865782684224, 3),
	4: (-26.57602457275058, 5),
	5: (-25.62409170860287, 5),
	6: (-16.504784428315524, 7),
	7: (-16.551627120896597, 7),
	8: (-8.705722911256922, 9),
	9: (-6.058270126581968, 9),
	10: (0, 0)
}
```

#### 5、未登录词处理  

jieba 分词使用 HMM 识别未登录词，根据语料统计 HMM 所需的参数，将上述最大概率中连续的单个字拼接起来，送入 HMM 分词模块中，进行进一步的分词，从而能够解决一部分未登录词的问题。

## 3、LSTM 分词  

- **这里介绍 FoolNLTK 的分词过程**

#### 1、训练分词模型  

处理语料数据，分词状态标注类别为 B、M、E、S，将处理好的句子及对应的标注送入双向 LSTM 模型中提取特征，接着经过全连接层得到句子中每个字符在四个类别上的概率值，然后送入 crf_loss 损失函数中，计算预测与标注之间的损失，不断的迭代训练网络模型参数。  

注意，这里在计算 crf_loss 时，隐式的在状态中增加了一个 start 状态，所以训练 crf  得到的转移矩阵是 $5 * 5$ 而不是 $4 * 4$的。

```
[
	[-3.7461855 2.526161 - 5.083622 2.531592 0.33619034]
	[2.7428856 - 4.246627 1.8435313 - 3.5215127 0.6473589]
	[2.4329855 - 4.8178263 2.0321524 - 4.7590933 - 0.56929415]
	[-3.8915057 2.5035102 - 4.8904557 2.6087193 - 0.14733171]
	[1.6312431 - 3.365449 0.8460278 - 3.0054417 0.4821391]
]
```

#### 2、分词过程  

将待分词句子送入训练好的分词模型中，计算得到句子中每个字符在四个状态类别上的概率值  

```
[
	[3.4461014 - 2.6352444 - 0.16313046 0.05346665]
	[-0.26584226 2.082723 - 1.9984283 0.5756328]
	[-0.1461018 3.558399 0.26490203 - 1.002711]
	[1.1721244 1.0938084 3.3237858 - 1.9899902]
	[3.3887694 - 0.72693497 - 0.08797821 - 0.689526]
	[0.03522667 1.659201 - 0.01005548 0.18743217]
	[3.794674 - 0.63267696 - 1.9391873 - 0.7672459]
	[-2.172909 3.1337032 2.2368207 0.38919255]
	[4.5226216 - 1.3995167 0.5482619 - 1.6732168]
	[-0.72521657 3.6698503 - 1.0884228 - 0.39269456]
]
```

在计算得到的概率值上增加 start 状态，然后进行 viterbi 解码  

```
small = -1000.0
start = np.asarray([[small] * tag_num + [0]])  # 在句子开始前增加一行 start 状态对应的概率值
pad = small * np.ones([seq_lenth, 1])          # 在句子中每个字符的概率值后增加 start 状态对应的概率值，基本上是最小的
score = np.concatenate([score, pad], axis=1)   # 拼接增加的概率值
score = np.concatenate([start, score], axis=0)
```

预测得到句子对应的状态结果及分词结果  

```
新华网驻东京记者报道

['B', 'M', 'E', 'S', 'B', 'E', 'B', 'E', 'B', 'E']

['新华网', '驻', '东京', '记者', '报道']
```

#### 3、支持自定义词典  

将自定义字典放到 trie tree 中，找出待分词句子中的存在的预定义的词，将通过词库找出的词和序列标注得到的词组合到一起，形成 DAG，通过后向匹配找到最大概率路径，得到最终的分词结果。  

### 命名实体识别和词性标注任务  

该模型同样可以用相同的计算过程完成对序列文本的命名实体识别和词性标注任务，只是训练模型用到的标注类别不同。  

- 命名实体识别标注：命名实体识别与分词的唯一区别就是每个字的标注类别不同，在该任务中每个字对应 25 个不同的类别，可以识别 6 种不同的实体，分别为 location，org，job，time，person，company

```
{
    0: 'O', 1: 'B_location', 2: 'M_location', 3: 'E_location', 4: 'B_org', 5: 'M_org', 6: 'E_org', 7: 'S_location',
    8: 'B_job', 9: 'E_job', 10: 'B_time', 11: 'M_time', 12: 'E_time', 13: 'B_person', 14: 'M_person', 15: 'E_person', 
    16: 'B_company', 17: 'M_company', 18: 'E_company', 19: 'S_person', 20: 'M_job', 21: 'S_time', 22: 'S_org', 
    23: 'S_job', 24: 'S_company'
}
```

- 词性标注：词性标注任务是在分词的基础上进行的，先对序列进行分词，然后判断每个词的词性，使用与分词相同的模型进行训练和测试，只是输入文本格式及类别不一样，可以将词性标注任务看做是在 word-level 的序列标注任务，词性标注共 78 个类别,每个类别对应不同的词性，分别为

```
{
    0: 'v', 1: 'n', 2: 'a', 3: 'c', 4: 'wt', 5: 'wm', 6: 'nz', 7: 'ns', 8: 'p', 9: 'b', 10: 'ude', 11: 'wd', 12: 'wj', 13: 'vshi',
    14: 'wn', 15: 'ad', 16: 'udeng', 17: 'm', 18: 'd', 19: 'wkz', 20: 'wky', 21: 'q', 22: 'vi', 23: 'vl', 24: 's', 25: 'r', 26: 't',
    27: 'wp', 28: 'f', 29: 'nl', 30: 'nr', 31: 'vyou', 32: 'k', 33: 'nt', 34: 'nr1', 35: 'nx', 36: 'w', 37: 'wyz', 38: 'wyy',
    39: 'wf', 40: 'h', 41: 'uzhe', 42: 'pbei', 43: 'nrf', 44: 'pba', 45: 'y', 46: 'ww', 47: 'z', 48: 'uguo', 49: 'usuo', 50: 'vd',
    51: 'an', 52: 'url', 53: 'wh', 54: 'uzhi', 55: 'bl', 56: 'dl', 57: 'al', 58: 'ws', 59: 'u', 60: 'email', 61: 'udh', 62: 'uyy',
    63: 'o', 64: 'vn', 65: 'ulian', 66: 'wb', 67: 'ip', 68: 'id', 69: 'nrs', 70: 'l', 71: 'n]nt', 72: 'vv',
    73: 'vf', 74: 'nnn', 75: 'i', 76: 'nn', 77: 'N'
}

```